# StyleGAN2-ADA-PyTorch-DR

## Preface
This StyleGAN2-ADA-PyTorch repository (including this Colab notebook) was forked from [Derrick Schultz](https://github.com/dvschultz/stylegan2-ada-pytorch), which was forked from [Nvidia's original repo](https://github.com/NVlabs/stylegan2-ada-pytorch). Additional comments and instructions added by Doug Rosman.

Many thanks to Derrick for developing these tools to make GANs accessible for artists and newcomers, and for generously sharing his knowledge and resources **for free**. Without artists/educators like this, we would be spending much more time (and money) getting these things working. If you find that you're using this Colab notebook extensively, and especially if you're using it to make work to display outside the classroom, **_please be sure to credit [Derrick Schultz](https://www.instagram.com/dvsmethid/?hl=en)_**, and I encourage you all to consider signing up for his [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send him a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz). Derrick also has a Slack channel dedicated to this stuff, which you can access as a Patreon subscriber (it's a good place to get some extra help).

~ ~ ~ ~ ~ ~ ~ ~ ~

This Notebook was last updated: March 4, 2023 by [Doug Rosman](https://dougrosman.com/).

Additional Colab and Python help from Blake Fall-Conroy.

~ ~ ~ ~ ~ ~ ~ ~ ~

# Part 1: Setup (required every time you run this notebook)


These setup steps are required to get your GPU instance running, mount your Google Drive, and install the necessary dependencies.

## 1.1 Connect to a GPU Instance (required if training or generating images)

**SKIP THIS STEP IF YOU ARE DOING NON-GPU RELATED TASKS**
This command will connect you to a GPU instance and randomly select a GPU. The GPU you get depends on the subscription tier you have.

- **A100** - amazing, Pro+ only, train/generate up to 1024x1024
- **V100** - great, Pro+ only, train/generate up to 1024x1024
- **P100** - solid, Pro+ only, train/generate up to 1024x1024
- **T4** - good, Free and Pro, train up to 512x512, generate up to 1024x1024
- **K80** - okay, Free, train* up to 512x512, generate up to 1024x1024

If you get a K80 when you're trying to train a model, I recommend terminating the session and starting again (hopefully you'll get a better GPU!). **To terminate your session, at the top of your screen go to Runtime --> Manage Sessions --> Terminate**

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 4090 (UUID: GPU-4cc36cd3-6aa8-06a2-ec0b-dee66b991388)


## 1.3 Initialize the StyleGAN2-ADA-PyTorch repository (required)

This clones my fork of the StyleGAN2-ada-PyTorch GitHub repository to your Google Drive. If the repository already exists, this command will `cd` (change directory) into the correct folder.

In [2]:
import os

!mkdir downloads
!mkdir datasets
!mkdir pretrained
!mkdir generated
!mkdir generated/projection

The syntax of the command is incorrect.


## 1.5 Install Dependencies (required if training or generating images)

This installs the required dependencies, and uninstalls some libraries that create conflicts

In [2]:
#GPU frontend
# !pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


#Downgrade Pytorch
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 opensimplex
!pip install tqdm
!pip install imageio-ffmpeg
!pip install click
!pip install requests
!pip install scipy
!pip install psutil

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp39-cp39-win_amd64.whl (3128.0 MB)
  Using cached https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp39-cp39-win_amd64.whl (2.5 MB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl.metadata (61 kB)
  Using cached pillow-10.2.0-cp39-cp39-win_amd64.whl.metadata (9.9 kB)
Using cached pillow-10.2.0-cp39-cp39-win_amd64.whl (2.6 MB)
Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Using cached timm-0.4.12-py3-none-any.whl.metadata (30 kB)
  Using cached ftfy-6.1.1-py3-none-any.whl.metadata (6.1 kB)
  Using cached ninja-1.10.2-py2.py3-none-win_amd64.whl.metadata (5.0 kB)
  Using cached opensimplex-0.4.5-py3-none-any.whl.metadata (10 kB)
Using cached timm-0.4.12-py3-none-any.whl (376 kB)
Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
Using cached ninja-1.10.2-py2.py3-none-win_amd64.whl (837 kB)
Using cached opensimple

## 1.6 Finding pretrained models

Training models from scratch takes a lot of time, so it's best to transfer learn from a pretrained model.

Download any of the models below, and then upload the .pkl file to the `pretrained` folder in the stylegan2-ada-pytorch folder in your Google Drive. Note: once you train your own models, you can use those as pretrained models to transfer learn from.

**Important considerations for picking a pretrained model**
1. The pretrained model must be the same dimensions as your data set (e.g., if your images are 512x512, your trained model must also be 512x512)
2. The pretrained model's config must match your training config. **Unless you know what you're doing, stick with models labeled config-f.**

**Lists of pretrained models to choose from**
- https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ (Nvidia's official models (ada version))
  - [FFHQ](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl) (1024) (Flickr Faces High Quality, realistic human faces)
  - [MetFaces](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl) (1024) (Faces from paintings in the Met Museum of Art Collection)
  - [AFHQ Wild](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqwild.pkl) (512) (Animal Faces High Quality, realistic wild animal faces)
  - [AFHQ Cat](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqcat.pkl) (512) (Animal Faces High Quality, realistic cat faces)
  - [AFHQ Dog](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/afhqdog.pkl) (512) (Animal Faces High Quality, realistic dog faces)
  - [BreCaHAD](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/brecahad.pkl) (512) (Breast Cancer microscopic biopsy images)

- https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/ (Nvidia's official models)
  - [Cat](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-cat-config-f.pkl) (256)
  - [Church](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-church-config-f.pkl) (256)
  - [Horse](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-horse-config-f.pkl) (256)

- https://github.com/justinpinkney/awesome-pretrained-stylegan2 (collection by Justin Pinkney. There are more there than the few listed below)
  - [FFHQ 512](https://mega.nz/#!eQdHkShY!8wyNKs343L7YUjwXlEg3cWjqK2g2EAIdYz5xbkPy3ng) (512) (Lower resolution version of the FFHQ model)
  - [Flowers](https://drive.google.com/uc?id=13onBTt6xVwKmYTRCFbONBTbSXFqeiVcK) (256)
  - [Anime Portraits](https://mega.nz/#!PeIi2ayb!xoRtjTXyXuvgDxSsSMn-cOh-Zux9493zqdxwVMaAzp4) (512)

# Part 2: Data Set Preparation

Prepare your images for training by uploading a zip folder of images, then resizing and cropping them to your desired dimensions. This notebook contains a script for batch resizing and cropping your images. That said, you should take time to refine and clean your data set before uploading it to use in this notebook.

## 2.1 Zipping up your images and uploading to Google Drive


Create a folder and name it based on your data set (don't put spaces in your filename), then put all your images into that folder

### Creating a zip file

#### Windows Users:
1. Right-click the folder and select **Send to** --> **Compressed (zipped) folder**
1. Upload the .zip file to the **datasets** folder in Google Drive (inside the colab-sg2-ada-pytorch folder in your Google Drive)

#### Mac Users:
Zipping files on a Mac with the native compression tool creates a file called ".DS_Store" inside the folder, which is annoying because it causes problems. These steps include a workaround.

##### Download and Install [Keka](https://www.keka.io/en/)
1. Download Keka ([click to download](https://d.keka.io/))
1. Install Keka by dragging and dropping it into your Applications folder
1. Open Keka
1. Make sure the box next to **'Exclude Mac resource forks'** is checked
1. Drag and draop your data set folder directly onto the open Keka app
1. It may ask you where to save your file. Select your desintation and compress your folder
1. Upload the .zip file to the **datasets** folder in Google Drive (inside the colab-sg2-ada-pytorch folder in your Google Drive)

![good zip with Keka](https://raw.githubusercontent.com/dougrosman/stylegan2-ada-pytorch/main/images/keka.png)



## 2.2 Processing your images (batch cropping + resizing)



In order to train, all your images must be the exact same dimensions, and must be RGB images. Whatever state your data set is in, this step will resize and auto-crop each image to ensure the training will work. If your data set is already cleaned up and ready to go, you should still do this step (it won't change anything that's already in the correct format).

Taking care to prepare your data is an important part of working with these tools, so your data set *should* be ready to go before uploading.

StyleGAN2 most easily supports the following square resolutions:
- 256x256
- 512x512
- 1024x1024

This process processes each image with these steps:
1. Convert image to RGB
2. Upscale or downscale image as needed
3. Center-crop image

In [3]:
# Set your variables, then run this cell (required).

width = 1024  # EDIT THIS to your desired dimension (256, 512 or 1024)
height = width
dataset = "morning_test" # EDIT THIS to the name of your data set file (without the .zip)

data_src = "./datasets/" + dataset + ".zip"
data_dest = "./datasets/" + dataset + "-" + str(width) + ".zip" 

In [7]:
# Run this to process your images
!python dataset_tool-png.py --source=$data_src --dest=$data_dest --width=$width --height=$height --transform="center-crop"

Error: Missing input file or directory: ./datasets/morning_test.zip


# Part 3: Training your model

## 3.0 Training Tips/FAQ

### Tips
1. The more images you have, the better your model, but the longer it will take to train
1. Training at smaller resolutions will be faster.
1. Consider training a low resolution (256x256) model on your data first to see if you like the results. If you do, start a new training at a higher resolution.
1. Your model might be "good enough" after a small amount of training (200 kimg or so), but training longer will refine the quality of your images. Aim for 1,000 kimgs if you can.

### FAQ

**How many images should I have in my data set?**

It totally depends on your artistic goals! More images of your subject matter will lead to more "accurate" results, but that might not be what you're going for. Let's say that for "accuracy", you should have at least 500 images, but you should really be aiming for 2,000-5,000 (or more!). The FFHQ data set (Flickr Faces High Quality) that makes photorealistic faces was trained on 70,000 faces!



## 3.1 Setting your variables

In [8]:
# Set the following variables to make sure you're training with the correct settings.

# EDIT THIS to the name of your data set file (without the .zip)
dataset = "morning_test"

# EDIT THIS to the size of your images (256, 512 or 1024)
size = 1024 

# EDIT THIS to the .pkl file you're transfer learning from. This file must be in the "pretrained" folder
resume_from = "metfaces.pkl"

# (OPTIONAL) EDIT THESE
snapshot_count = 2  # set how frequently you save checkpoints. Lower == more frequent
mirror_x = False # set to 'True' to double your data set size by duplicating and mirroring each image horizontally. 
mirror_y = False # same as above, but vertically


# (DON'T CHANGE THESE UNLESS YOU KNOW WHAT YOU'RE DOING)
gamma_value = 50.0
augs = 'bg'
# config = '11gb-gpu'
config = '24gb-gpu'

####
dataset_path = "./datasets/" + dataset + "-" + str(size) + ".zip"
output_dir = "./results"
resume_from = "./pretrained/" + resume_from
aug_strength = 0
train_count = 0 

## 3.2 Setting your variables for resuming training (optional)



Only edit and run this cell if you are resuming a training your already started. You must edit and run the cell in 3.1 first.

* `resume_from`: The .pkl file you want to continue training from. This will probably be your latest .pkl file. You'll find this in your **results** folder.
* `aug_strength`: Set this to the augmentation amount for the pickle file you're resuming from. You can find this value in the "log.txt" file inside the results folder from the previous training. The number you're looking for will be in the right-most column of your log (**e.g. augment 0.652**). Select the augment value for the .pkl value you'll be training from. The number of kimgs will match the .pkl file.
* `train_count`: set this number to be the number at the end of the .pkl file you're resuming your training from. For example, if you're resuming your training from *network-snapshot-000200.pkl*, then set train_count to **200**.

In [ ]:
# ONLY DO THIS IF RESUMING FROM A TRAINING YOU'VE ALREADY STARTED
# Set the following arguments if you are resuming a training, then run this cell to save them
# EDIT THIS to the filepath of the .pkl you're resuming from
resume_from = './results/00001-your_dataset-512-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000020.pkl'
aug_strength = 0.003  # EDIT THIS TO YOUR CORRECT VALUE
train_count = 20  # EDIT THIS TO YOUR CORRECT VALUE

## 3.3 Start Training


This is it, the moment we've been waiting for! Once you start your training, remember to keep this tab open and your computer on (make sure you computer doesn't fall asleep. If you have Pro+, I believe you can close the tab and training will still continue.)

In [9]:
# After setting your variables above, run this cell to start your training.
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=$output_dir --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

# Part 4: Generating Images and Videos

For an in-depth explanation of the latent space, interpolations, and the different ways to generate latent space walks, check out this video: [StyleGAN2 In-Depth Week 3 (latent spaces, linear interpolations and noise loops](https://www.youtube.com/watch?v=jKJCv9VGqLQ&t=1998s) by Artificial Images (Derrick Schultz). The link takes you to the part of the video where he begins to go through these. (33:18) 

## 4.0 Key Terms

- `network` - The pickle file to generate from. Network is another term for "trained model."
- `seed` - Our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. Can be a number between 0 to 4,294,967,295. 
- `truncation` - How much to "truncate" the latent space. A measure of how "creative" or "realistic" the generator will be. Best results between -1 and 1. Low truncation (close to 0) produces a lower variety of images, adhering to the "average" image of your trained model. Higher truncation (close to 1) produces more varied images. Beyond -1 or 1, the results start getting more wonky, as these images are outside the bounds of the latent space.
- `interpolation` - Interpolation is a general concept of moving between two things. A trained model is a 512-dimension space, and *interpolating through* this space takes us through a range of images. There are different interpolation methods, each of which creates different "routes" through the latent space.

## 4.1 Single random images

This generates individual images for each seed value. This can be useful for finding a good starting image for the interpolation loops.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: How much truncation, a number between -1 to 1 will produce the best results, but technically can be infinite. 

`--seeds`: The seeds to generate images from. Can use a range of values (e.g. 400-500), or comma-separated seed values (e.g. 10, 20, 30).

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).


In [ ]:
!python generate.py \
--outdir=./generated/random-1000-1020-noncon \
--trunc=1 \
--seeds=1000-1020 \
--network=./pretrained/cat256.pkl

## 4.2 Truncation Traversal



Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

`--network`: Again, this should be the path to your .pkl file.

In [ ]:
!python generate.py \
--process="truncation" \
--outdir=./generated/trunc-1002_1004__-2-2 \
--start=-2 \
--stop=2 \
--increment=0.005 \
--seeds=1002 \
--network=./pretrained/cat256.pkl \
--fps=60

## 4.3 Linear Interpolation (Lerp)


A straight line from one point (seed) in the latent space to another. The distance from one seed to another will vary, so some interpolations will be fast, others slow.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--space`: "z" or "w". Depending on your model, w-space may interpolate more smoothly (like with faces).

`--trunc`: truncation value

`--seeds`: list of seeds to interpolate between. Comma separated. Each set of seeds will be given the same number of frames to complete the interpolation. Set the last seed to the same number as the first to create a seamless looping video.

`--network`: path to your .pkl file

`--frames`: How many frames to produce per interpolation. (e.g. if you set your frames to **30** and your seeds to 10, 20, 30, 10, this will create a video with **90** frames (10 to 20 (30 frames), 20 to 30 (30 frames), 30 to 10 (30 frames))

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [ ]:
!python generate.py \
--outdir=./generated/lerp-10_20_30_40_10-w2 \
--space="w" \
--trunc=0.75 \
--process="interpolation" \
--seeds=10,20,30,40,10 \
--network=./pretrained/cat256.pkl \
--frames=120 \
--fps=60

## 4.4 Noise Loop


If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path through the z space to show you a diverse set of images.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: truncation value

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

`--network`: path to your .pkl file

`--frames`: How many frames to produce

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [ ]:
!python generate.py \
--outdir=./generated/noiseloop_1003_d1 \
--trunc=0.75 \
--process="interpolation" \
--interpolation="noiseloop" \
--diameter=1 \
--random_seed=1003 \
--network=./pretrained/cat256.pkl \
--frames=120 \
--fps=60

## 4.5 Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: truncation value

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

`--network`: path to your .pkl file

`--frames`: How many frames to produce

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [ ]:
!python generate.py \
--outdir=./generated/circularloop_1003_d1000 \
--trunc=0.75 \
--process="interpolation" \
--interpolation="circularloop" \
--diameter=1000 \
--random_seed=1003 \
--network=./pretrained/cat256.pkl \
--frames=120 \
--fps=60

# Part 5: Projection


Project images into the latent space. The network looks at your image and tries to reverse engineer its latent vector in order to re-create your image from what the latent space understands about images.

**Steps:**
1. Find an image you want to project
2. Resize it so that it is the same dimensions as your trained model
3. Upload the image to your Google Drive in the stylegan2-ada-pytorch repository (in the generated/project/input folder)

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--target`: Filepath to your input image

`--num-steps`: how many iterations the projector should run for. Lower will mean fewer steps and less likelihood of a good projection. Higher will take longer but will likely produce better images.

`--network`: Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).


In [ ]:
!python projector.py \
--outdir=./generated/projection/lea6 \
--target=./generated/projection/input/lea256.jpg \
--num-steps=30 \
--seed=20 \
--network=./pretrained/cat256.pkl \
--video-target-concat=True # set to True for side-by-side of target image + output video

# Part 6: Face Alignment

*Work in progress, eventually this script will accept a .zip folder.*

Align faces using the same process used to align faces for the FFHQ data set. Run this script if you plan to train a data set on faces, or project a face into a latent space with aligned faces like FFHQ, metfaces, or AFHQ (Wild, Dog or Cat) Although this should maybe go in the Data Set Preparation section, I'm including it here at the bottom since it's a more of a niche process that won't be run that frequently.

**Steps**
(again, this is a work in progress, so the process is a little less streamlined)




In [ ]:
# 1. change directory into the align_faces folder
%cd util/align_faces/

In [ ]:
# 2. "clean" your images by making sure the alignment script can process them
!python clean_images_downscale.py /path/to/dataset.zip 2000

In [ ]:
# 3. unzip your images to a folder called raw_images
!unzip output.zip -d raw_images

In [ ]:
# 4. align the faces, saving them to a folder called aligned
!python align_faces.py raw_faces aligned

In [ ]:
#5. (optional) zip up the aligned faces so they can be used for training
!zip -r aligned.zip aligned

In [ ]:
# change directory back into the main folder so that all the other scripts work
%cd /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch

# Convert Legacy Model



If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [ ]:
!python legacy.py --source=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/cat256.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/cat256-converted.pkl

## Combine NPZ files together

In [ ]:
!python combine_npz.py --outdir=/content/npz --npzs='/content/pb-proj1-3-clip.npz,/content/pb-proj1-clip-nocenter.npz,/content/pb-proj2-clip-nocenter.npz,/content/projected_w (2).npz'

# Random snippets

In [ ]:
!zip -r vid1.zip /content/out/video1-w-0.5

In [ ]:
!ffmpeg -y -r 144 -i ./generated/ffhq-144fps/frames/frame%04d.png -vcodec libx265 -crf 25 -preset faster -pix_fmt yuv420p ./generated/ffhq-144fps/lerp144.mp4

#### 4.4 Spherical Interpolation (Slerp) (Not Working)

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations and discussions.

That said, the differences between lerp and slerp aren't all that significant.

Note: Slerp in w space currently isn’t supported.

**Options**

`--outdir`: Output directory (folder) where your images will be saved.

`--trunc`: truncation value

`--seeds`: list of seeds to interpolate between. Comma separated. Each set of seeds will be given the same number of frames to complete the interpolation. Set the last seed to the same number as the first to create a seamless looping video.

`--network`: path to your .pkl file

`--frames`: How many frames to produce per interpolation. (e.g. if you set your frames to **30** and your seeds to 10, 20, 30, 10, this will create a video with **90** frames (10 to 20 (30 frames), 20 to 30 (30 frames), 30 to 10 (30 frames))

`--fps`: frames per second of your video (24, 30, 48, 60, 120, 144)

In [ ]:
!python generate.py \
--outdir=./generated/slerp-10_20_30_40_10-z \
--interpolation="slerp" \
--process="interpolation" \
--trunc=0.75 \
--seeds=10,20,30,40,10 \
--network=./pretrained/cat256.pkl \
--frames=120 \
--fps=60